In [21]:
import pandas as pd
import sqlite3

In [22]:
group1_df = pd.read_csv("./Group_1.csv")

In [23]:
group2_df = pd.read_csv("./Group_2.csv")

In [24]:
# create connection to in memory sqlite db
conn = sqlite3.connect(":memory:")

# save data-frame to table 
group1_df.to_sql(name="group_1_txt", con=conn, index=False)
group2_df.to_sql(name="group_2_txt", con=conn, index=False)

In [25]:
query1 = """

with Group1_Enrich as (
select 
  PersonID, 
  CAST(Height_CM as decimal) as Height,
  DATE(DOB) as BirthDate
from group_1_txt
),

Group2_Enrich as (
select 
  PersonID, 
  CAST(Height_CM as decimal) as Height, 
  DATE(DOB) as BirthDate
from group_2_txt
),

Group1_Min_BirthDate as (
select 
    min(BirthDate) as Birth_Date,
    'Group1' as GroupName,
    PersonID,
    Height,
    'Eldest' as Remarks
from Group1_Enrich
),

Group1_Max_Height as (
select 
  BirthDate as Birth_Date,
  'Group1' as GroupName,
   PersonID,
  max(Height) as Height,
  'Tallest' as Remarks
from Group1_Enrich
),

Group2_Min_BirthDate as (
select 
   min(BirthDate) as Birth_Date,
   'Group2' as GroupName,
   PersonID,
   Height,
   'Eldest' as Remarks
 from Group2_Enrich
),

Group2_Max_Height as (
select 
  BirthDate as Birth_Date,
  'Group2' as GroupName,
  PersonID,
  max(Height) as Height,
  'Tallest' as Remarks
  from Group2_Enrich
)

---- Return the final result 
select 
*
from 
 Group1_Min_BirthDate
union all
select 
*
from 
  Group1_Max_Height
union all
select 
*
from 
  Group2_Min_BirthDate
union all
select 
*
from   
  Group2_Max_Height
  
  
"""

In [26]:
# use pandas read_sql to execute the query and return a dataframe
result1_df = pd.read_sql(query1, con=conn)

In [27]:
result1_df

,Birth_Date,GroupName,PersonID,Height,Remarks
0,1976-10-11,Group1,21821,175,Eldest
1,1976-10-11,Group1,21821,175,Tallest
2,1993-10-11,Group2,21824,175,Eldest
3,1993-10-11,Group2,21824,175,Tallest
